# Aplicaciones a Problemas Inversos

**Por:** David Ortiz, Rodrigo Salas  
**Edición:** David Ortiz, Tabita Catalán, Tomás Banduc  

Referencia sobre estimación de parámetros en problemas inversos:  
https://arxiv.org/abs/2308.00927

## Introducción

En esta actividad se estudia el uso de PINNs para **problemas inversos**, extendiendo el enfoque aplicado previamente a problemas directos. El objetivo es estimar parámetros desconocidos de un modelo físico a partir de observaciones ruidosas. En particular, se abordará la estimación del **coeficiente de difusión** $\kappa$ en un modelo de difusión lineal.

## Resumen de la actividad

Se implementa una PINN para estimar el coeficiente de difusión del modelo de calor unidimensional, tratando $\kappa$ como un parámetro entrenable junto con la solución del campo $u(t,x)$.

## Objetivos

Al finalizar esta actividad, serás capaz de:
- Aplicar PINNs a la resolución de problemas inversos.  
- Entrenar una PINN para resolver una EDP y estimar simultáneamente un parámetro desconocido del modelo.

## Descripción matemática del problema

Consideramos nuevamente el modelo de difusión unidimensional:

$$
\begin{aligned}
\text{EDP:}\quad &
\frac{\partial u}{\partial t}
= \kappa \frac{\partial^2 u}{\partial x^2}
- e^{-t}\big(\sin(\pi x) - \pi^2 \sin(\pi x)\big),
&& x \in [-1,1],\; t \in [0,2], \\
\text{Solución:}\quad &
u(t,x) = e^{-t}\sin(\pi x).
\end{aligned}
$$

Aquí $u(t,x)$ representa una cantidad física de interés y $\kappa$ es el coeficiente de difusión, tratado como **parámetro desconocido** durante el entrenamiento. En este caso, el valor verdadero es $\kappa = 1$, que la PINN debe inferir a partir de los datos.

A diferencia de los problemas directos, el entrenamiento se apoya en **observaciones ruidosas** $u_{\text{data}}(t,x)$ de la solución analítica, que aportan la información necesaria para identificar el parámetro.

## Flujo de trabajo

1. Generar observaciones ruidosas de la solución analítica.  
2. Muestrear el dominio espacio–temporal para el entrenamiento.  
3. Entrenar la PINN para resolver la EDP y estimar simultáneamente el parámetro $\kappa$.  

## Recordemos el esquema básico
<img src="../figures/pinns_new_scheme.png" width="800" height="400">

- (1) formular el modelo matemático y su solución analítica;  
- (2) definir el dominio espacio–temporal;  
- (3) implementar una ANN como aproximador de $u(t,x)$;  
- (4) emplear diferenciación automática;  
- (5) diseñar la *loss* con términos físicos y de datos usando `autograd`;  
- (6) seleccionar un optimizador (Adam).  


### Configuración Inicial

Comenzamos importando módulos y definiendo algunas funciones de utilidad

In [ ]:
%matplotlib inline

In [ ]:
# NumPy para operaciones numéricas
import numpy as np
# PyTorch para construir y entrenar redes neuronales
import torch
import torch.nn as nn
import torch.optim as optim
# Matplotlib para graficar
import matplotlib.pyplot as plt
import matplotlib as mpl
# Time para medir tiempo de entrenamiento
import time
# Warnings para ignorar mensajes de advertencia
import warnings
warnings.filterwarnings("ignore")

from matplotlib import animation, rc
from scipy.stats import qmc

In [ ]:
# Setup (device + plots)
def get_device() -> str:
    return "cuda" if torch.cuda.is_available() else \
           "mps"  if torch.backends.mps.is_available() else "cpu"

def set_mpl_style(gray: str = "#5c5c5c") -> None:
    mpl.rcParams.update({
        "image.cmap": "viridis",
        "text.color": gray, "xtick.color": gray, "ytick.color": gray,
        "axes.labelcolor": gray, "axes.edgecolor": gray,
        "axes.spines.right": False, "axes.spines.top": False,
        "axes.formatter.use_mathtext": True, "axes.unicode_minus": False,
        "font.size": 15, "interactive": False, "font.family": "sans-serif",
        "legend.loc": "best", "text.usetex": False, "mathtext.fontset": "stix",
    })

device = get_device()
print(f"Using {device} device")
set_mpl_style()

# Definir pi en torch
torch.pi = torch.acos(torch.zeros(1)).item() * 2

def relative_l2_error(u_num: torch.Tensor, u_ref: torch.Tensor) -> torch.Tensor:
    return torch.norm(u_num - u_ref) / torch.norm(u_ref)

# Autodiff helper
def grad(outputs: torch.Tensor, inputs: torch.Tensor) -> torch.Tensor:
    """d(outputs)/d(inputs) with create_graph=True."""
    return torch.autograd.grad(
        outputs, inputs,
        grad_outputs=torch.ones_like(outputs),
        create_graph=True
    )[0]

def plot_comparison(u_true, u_pred, loss, k_evol):
    u_hat = u_pred.detach().cpu().numpy()

    # --- Soluciones ---
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    for a, u, title in zip(
        ax,
        [u_true, u_hat],
        ['Analytic solution', 'PINN solution']
    ):
        im = a.imshow(u, extent=[-1, 1, 2, 0])
        a.set(title=title, xlabel=r'$x$', ylabel=r'$t$')
        fig.colorbar(im, ax=a, shrink=0.5)

    fig.tight_layout()
    plt.show()

    # --- Entrenamiento ---
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    ax[0].plot(k_evol, label='PINN estimate')
    ax[0].hlines(1, 0, len(k_evol), label='True value')
    ax[0].set(title=r'$\kappa$ evolution', xlabel='Iteration')
    ax[0].legend(frameon=False)

    ax[1].plot(loss)
    ax[1].set(title='Training Progress',
              xlabel='Iteration', ylabel='Loss',
              xscale='log', yscale='log')
    ax[1].grid(True)

    fig.tight_layout()
    plt.show()

def animate(x,t,U):
    # Primero, generar figura con subplot correspondiente
    fig, ax = plt.subplots(figsize = (10,6))
    plt.close()
    ax.set_title(r"heat solution $e^t sin(\pi x)$")
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(np.floor(U.min()), np.ceil(U.max()))
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"$u(x, t)$")
    # Inicializar etiqueta sin texto
    time_label = ax.text(1, 1, "", color = "black", fontsize = 12)
    # Inicializar gráfico sin datos
    line, = ax.plot([], [], color = "black", lw = 2)

    # Definir función de inicialización
    def init():
        line.set_data([], [])
        time_label.set_text("")
        return (line,)

    # Animar función. Esta función se llama secuencialmente con FuncAnimation
    def animate(i):
        line.set_data(x, U[i])
        time_label.set_text(f"t = {t[i]:.2f}")
        return (line,)

    return animation.FuncAnimation(fig, animate, init_func=init,
                                frames=len(t), interval=50, blit=True)


## 1. Solución Analítica
Una vez más, definimos la solución analítica $u(t,x) = e^{-t}\sin(\pi x)$, que será considerada la solución pura del problema para efectos de comparación. Para esta tarea, `analytic_diffusion` será usada para generar las observaciones de dicha función.

In [ ]:
# Número de muestras para espacio y tiempo.
dom_samples = 100

# Función para solución analítica
def analytic_diffusion(x,t):
    u = np.exp(-t)*np.sin(np.pi*x)
    return u

# Dominio espacial
x = np.linspace(-1, 1, dom_samples)
# Dominio temporal
t = np.linspace(0, 2, dom_samples)

# Mallado
X, T = np.meshgrid(x, t)
# Evaluar función en mallado
U = analytic_diffusion(X, T)

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, T, U, cmap='viridis', edgecolor='k')

ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_zlabel('u(t, x)')
ax.set_title('3D Analytic Solution for Diffusion')

# Añadir la barra de color
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.show()

In [ ]:
# Correr animación y hacer display
anim = animate(x,t,U)
rc("animation", html="jshtml")
anim

## 2. Muestreo del Dominio para entrenar la PINN
Para entrenar la PINN, aplicaremos la estrategia LHS (*Latin Hypercube Sampling*). Hay que recordar que LHS asegura que las muestras cubran uniformemente el espacio de entrada, previniendo la clusterización en pequeñas áreas del dominio.

Importamos `qmc.LatinHypercube` de `scipy.stats` y escalamos las muestras para adaptarlas al borde del dominio. Adicionalmente, convertimos el dominio temporal y las observaciones a `torch.tensors` para compatibilizarlas con la PINN.

In [ ]:
from scipy.stats import qmc
# Muestreo con LHS
sampler = qmc.LatinHypercube(d=2)
sample = sampler.random(n=100)

# Límites del dominio
l_bounds = [-1, 0]
u_bounds = [ 1, 2]
domain_xt = qmc.scale(sample, l_bounds, u_bounds)

# Tensores de torch
x_ten = torch.tensor(domain_xt[:, 0], requires_grad = True).float().reshape(-1,1)
t_ten = torch.tensor(domain_xt[:, 1], requires_grad = True).float().reshape(-1,1)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(domain_xt[:, 0],domain_xt[:, 1], label = 'PDE collocation points')
ax.set_title('Collocation points')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$t$')
ax.legend(loc='lower left')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


Además, evaluamos `analytic_diffusion` en los puntos de colocación. La inclusión de ruido se sigue del supuesto que las observaciones vienen dadas de la superposición entre la solución analítica y alguna variable aleatoria (típicamente gaussiana) generada por defecto durante la medición, i.e., $u_{data}(t,x)=u(t,x)+\varepsilon$, con $\varepsilon\sim \mathcal{N}(0,\sigma)$.

In [ ]:
# Evaluar puntos en función analítica
x_np = x_ten.detach().numpy()
t_np = t_ten.detach().numpy()
u_true = analytic_diffusion(x_np,t_np).reshape(1, -1)
u_observ = u_true + np.random.normal(0,0.01,len(x_np))
# Convertir observaciones a torch
u_observ_t = torch.tensor(u_observ, requires_grad = True).float().reshape(-1,1)

## 3. Solución del problema inverso con PINNs

En este apartado se aborda la estimación del parámetro de difusión $\kappa$ de forma simultánea al entrenamiento de la PINN. Para ello, la red neuronal aproxima directamente la solución de la EDP y se introduce $\kappa$ como un **parámetro entrenable adicional**:

$$
u_{\text{PINN}}(t,x;\Theta,\kappa) \approx u(t,x),
$$

donde $\Theta$ denota los parámetros de la red y $\kappa$ el parámetro físico desconocido. La implementación se realiza en `PyTorch`, manteniendo la misma arquitectura utilizada en los problemas directos.

> **💡 Nota**  
> En problemas inversos, los parámetros físicos desconocidos (como $\kappa$) se tratan como variables entrenables y deben incorporarse explícitamente al proceso de optimización.

### Función de pérdida informada por la física

Para entrenar la PINN se define el residuo de la ecuación de calor, reemplazando la solución exacta por la salida de la red:

$$
f_{\text{pde}}(t,x;u_{\text{PINN}}) =
\frac{\partial u_{\text{PINN}}}{\partial t}
- \kappa \frac{\partial^2 u_{\text{PINN}}}{\partial x^2}
+ e^{-t}\big(\sin(\pi x) - \pi^2 \sin(\pi x)\big).
$$

> **💡 Nota**  
> Al tratarse de un problema inverso, no se imponen condiciones iniciales ni de borde; la información necesaria proviene de los datos observados.

La función de pérdida se construye mediante el error cuadrático medio (MSE), combinando el residuo físico y el ajuste a los datos:

$$
\begin{aligned}
\mathcal{L}(\Theta,\kappa) =\;&
\frac{\lambda_1}{N}\sum_i f_{\text{pde}}(t_i,x_i;u_{\text{PINN}})^2 \\
&+ \frac{\lambda_2}{N}\sum_i
\big(u_{\text{PINN}}(t_i,x_i;\Theta,\kappa) - u_{\text{data}}(t_i,x_i)\big)^2,
\end{aligned}
$$

donde $\lambda_{1,2} \in \mathbb{R}^+$ son coeficientes de ponderación y $N$ es el número de muestras.

> **💡 Nota**  
> Un problema inverso se aborda mediante un esquema *data-driven*, ya que la estimación del parámetro requiere información proveniente de observaciones.

El entrenamiento se formula como el problema de optimización

$$
\min_{\Theta,\kappa} \; \mathcal{L}(\Theta,\kappa) \rightarrow 0.
$$

### Definición del optimizador

Una vez definida la función de pérdida, se selecciona un optimizador para ajustar **tanto** los parámetros de la red $\Theta = \{W_i,b_i\}$ **como** el parámetro físico $\kappa$. El objetivo es minimizar la función de pérdida informada por la física y los datos mediante un esquema iterativo:

$$
\begin{aligned}
\Theta^{k+1} &= \Theta^{k} - \alpha \nabla_{\Theta}\mathcal{L}(\Theta^{k},\kappa^{k}), \\
\kappa^{k+1} &= \kappa^{k} - \alpha \nabla_{\kappa}\mathcal{L}(\Theta^{k},\kappa^{k}).
\end{aligned}
$$

En la práctica, ambos conjuntos de parámetros se incluyen conjuntamente en el optimizador (por ejemplo, **Adam**), permitiendo que la PINN aprenda simultáneamente la solución de la EDP y el valor del coeficiente de difusión.


In [ ]:
# Definir clase de red neuronal con capas y neuronas especificadas por usuario
class NeuralNetwork(nn.Module):

    def __init__(self, hlayers):
        super(NeuralNetwork, self).__init__()

        layers = []
        for i in range(len(hlayers[:-2])):
            layers.append(nn.Linear(hlayers[i], hlayers[i+1]))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hlayers[-2], hlayers[-1]))

        self.layers = nn.Sequential(*layers)
        self.init_params()

    def init_params(self):
        """Inicialización de parámetros Xavier Glorot
        """
        def init_normal(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight) # Xavier
        self.apply(init_normal)

    def forward(self, x):
        return self.layers(x)

#===============================================================================
# ETAPA 3: CREACIÓN DE LA RED NEURONAL SURROGANTE
#===============================================================================
torch.manual_seed(123)

# hiper-parámetros de la red
hidden_layers = [2, 10, 10, 10, 1]

# Crear instancia de la NN
u_pinn = NeuralNetwork(hidden_layers)
nparams = sum(p.numel() for p in u_pinn.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

#==========================================================================
# ETAPA 6: DEFINICIÓN DEl OPTIMIZADOR
#==========================================================================
learning_rate = 0.001

# Tratar k como parámetro entrenable
kappa = torch.nn.Parameter(torch.ones(1, requires_grad=True)*2)
kappas = []

# Definir optimizador y agregar k
optimizer = optim.Adam(list(u_pinn.parameters())+[kappa], lr=0.001,
                       betas= (0.99,0.999), eps = 1e-8)


A continuación se presenta el código completo

In [ ]:
#===============================================================================
# ETAPA 1: DEFINICIÓN DE LOS PARÁMETROS (MODELO FÍSICO)
#===============================================================================
from scipy.stats import qmc
# Muestreo con LHS
sampler = qmc.LatinHypercube(d=2)
sample = sampler.random(n=100)

# Límites del dominio
l_bounds = [-1, 0]
u_bounds = [ 1, 2]
domain_xt = qmc.scale(sample, l_bounds, u_bounds)

# Función para solución analítica
def analytic_diffusion(x,t):
    u = np.exp(-t)*np.sin(np.pi*x)
    return u

# Evaluar puntos en función analítica
x_np = domain_xt[:, 0]
t_np = domain_xt[:, 1]
u_true = analytic_diffusion(x_np,t_np).reshape(1, -1)
u_observ = u_true + np.random.normal(0,0.01,len(x_np))
# Convertir observaciones a torch
u_observ_t = torch.tensor(u_observ, requires_grad = True).float().reshape(-1,1)

#===============================================================================
# ETAPA 2: DEFINICIÓN DEL DOMINIO
#===============================================================================
# Tensores de torch
x_ten = torch.tensor(domain_xt[:, 0], requires_grad = True).float().reshape(-1,1)
t_ten = torch.tensor(domain_xt[:, 1], requires_grad = True).float().reshape(-1,1)

#===============================================================================
# ETAPA 3: CREACIÓN DE LA RED NEURONAL SURROGANTE
#===============================================================================
torch.manual_seed(123)

# hiper-parámetros de la red
hidden_layers = [2, 10, 10, 10, 1]

# Crear instancia de la NN
u_pinn = NeuralNetwork(hidden_layers)
nparams = sum(p.numel() for p in u_pinn.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

#==========================================================================
# ETAPA 4 Y 5: DEFINICIÓN DE LA FUNCIÓN DE COSTO BASADA EN AUTOGRAD
#==========================================================================
# Error cuadrático medio (Mean Squared Error - MSE)
MSE_func = nn.MSELoss()

def PINN_diffusion_Loss(forward_pass, x_ten, t_ten, kappa,
             lambda1 = 1, lambda2 = 1):

    # define domain
    domain = torch.cat([t_ten, x_ten], dim = 1)

    # define the ANN output
    u = forward_pass(domain)

    # TODO: compute the first derivative in time, and first and second derivatives in space
    u_t = ...
    u_x = ...
    u_xx = ...

    # TODO: define the PDE loss
    f_pde = ...
    PDE_loss = lambda1 * ...

    # TODO define the data loss
    data_loss = lambda2 * ... # you should use u_observ_t

    return PDE_loss + data_loss

#==========================================================================
# ETAPA 6: DEFINICIÓN DEl OPTIMIZADOR
#==========================================================================
learning_rate = 0.001

# Tratar k como parámetro entrenable
kappa = torch.nn.Parameter(torch.ones(1, requires_grad=True)*2)
kappas = []

# Definir optimizador y agregar k
optimizer = optim.Adam(list(u_pinn.parameters())+[kappa], lr=0.001,
                       betas= (0.99,0.999), eps = 1e-8)


#==========================================================================
# CICLO DE ENTRENAMIENTO
#==========================================================================

training_iter = 25000

# Inicializar lista para guardar valores de pérdida
loss_values = []

# Empezar timer
start_time = time.time()

# Entrenar red neuronal
for i in range(training_iter):

    optimizer.zero_grad()   # Reinicializar gradientes para iteración de entrenamiento

    # ingresar x, predecir con PINN y obtener pérdida
    loss = PINN_diffusion_Loss(u_pinn, x_ten, t_ten, kappa)

    # Agregar actual valor de pérdida a la lista y agregar valor de kappa actual
    loss_values.append(loss.item())
    kappas.append(kappa.item())

    if i % 1000 == 0:  # Imprimir pérdida cada 1000 iteraciones
        print(f"Iteration {i}: Loss {loss.item()}")

    loss.backward() # Paso de retropropagación
    optimizer.step() # Actualizar pesos de la red con optimizador

# Detener timer y obtener tiempo transcurrido
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

In [ ]:
dom_samples = 100

# Dominio espacial
x = np.linspace(-1, 1, dom_samples)
# Dominio temporal
t = np.linspace(0, 2, dom_samples)

# Mallado
X, T = np.meshgrid(x, t)

# Evaluar función en mallado
U = analytic_diffusion(X, T)

X_ten = torch.tensor(X).float().reshape(-1, 1)
T_ten = torch.tensor(T).float().reshape(-1, 1)
domain_ten = torch.cat([T_ten, X_ten], dim = 1)
U_pred = u_pinn(domain_ten).reshape(dom_samples,dom_samples)

U_true = torch.tensor(U).float()
print(f'Relative error: {relative_l2_error(U_pred, U_true)}')

plot_comparison(U, U_pred, loss_values, kappas)

**Ejercicios**:
1. Disminuya y aumente el valor inicial del parámetro $\kappa$. Considere que en varias aplicaciones solo se conoce un intervalo de búsqueda para los parámetros desconocidos.
2. Evalúe cómo varía la solución de la PINN aumentando y disminuyendo los pesos `lambdas`.
3. Evalúe cómo varía la solución de la PINN aumentando y disminuyendo la tasa de aprendizaje y el número de iteraciones de entrenamiento.
4. Cambie el número de capas ocultas, neuronas y funciones de activación del modelo de NN.
